# Average Scaled MinHash sketch sizes across Genbank domains

scaled = 2000

In [1]:
from functools import reduce
import operator

import pandas as pd
import hvplot.pandas

pd.options.plotting.backend = "hvplot"

In [2]:
# domain = "protozoa"
domain = "archaea"
# domain = "fungi"

In [3]:
df = pd.read_csv(f"/data/databases/{domain}.csv").set_index("id")

In [4]:
df.head()

,21,unique_21,31,unique_31,51,unique_51,bp
id,,,,,,,
GCA_900103715.1,3,5031,3,4508,3,3430,6968
GCA_000220355.1,3,7557,4,7177,1,6364,8650
GCA_007279865.1,261,526483,255,518713,266,526684,525666
GCA_011054285.1,181,413227,220,415898,205,414352,422932
GCA_003702585.1,271,525641,267,523830,269,536773,534624


In [5]:
df.describe()

,21,unique_21,31,unique_31,51,unique_51,bp
count,5234.000000,5.234000e+03,5234.000000,5.234000e+03,5234.000000,5.234000e+03,5.234000e+03
mean,767.079671,1.538620e+06,766.188193,1.540598e+06,766.778945,1.543335e+06,1.563439e+06
std,575.228923,1.152912e+06,577.762985,1.154493e+06,579.626693,1.159104e+06,1.174559e+06
min,0.000000,1.240000e+02,0.000000,1.030000e+02,0.000000,6.400000e+01,1.640000e+02
25%,372.250000,7.534760e+05,376.000000,7.515648e+05,374.250000,7.543002e+05,7.643212e+05
50%,718.000000,1.440550e+06,708.000000,1.440841e+06,710.000000,1.439552e+06,1.459127e+06
75%,1052.750000,2.102861e+06,1056.000000,2.112164e+06,1047.000000,2.117172e+06,2.133191e+06
max,6476.000000,1.325708e+07,6684.000000,1.305429e+07,6670.000000,1.315746e+07,1.339992e+07


In [6]:
df[["21", "31", "51"]].hvplot.box(xlabel="k-size", ylabel="sketch size")

:BoxWhisker   [Variable]   (value)

In [7]:
df

,21,unique_21,31,unique_31,51,unique_51,bp
id,,,,,,,
GCA_900103715.1,3,5031,3,4508,3,3430,6968
GCA_000220355.1,3,7557,4,7177,1,6364,8650
GCA_007279865.1,261,526483,255,518713,266,526684,525666
GCA_011054285.1,181,413227,220,415898,205,414352,422932
GCA_003702585.1,271,525641,267,523830,269,536773,534624
...,...,...,...,...,...,...,...
GCA_002499195.1,859,1838236,913,1835197,928,1827482,1853906
GCA_009762275.1,1789,3478100,1773,3556322,1741,3529795,3560302
GCA_003157355.1,804,1636340,841,1619918,763,1638426,1640806


In [8]:
plots = []
for k in ("21", "31", "51"):
    plots += [df.hvplot.scatter(y=k, x="bp")]
reduce(operator.add, plots).cols(1)

:Layout
   .Scatter.I   :Scatter   [bp]   (21)
   .Scatter.II  :Scatter   [bp]   (31)
   .Scatter.III :Scatter   [bp]   (51)

In [9]:
plots = []
for k in ("21", "31", "51"):
    df[f"constant_{k}"] = df[f"unique_{k}"] / 2000
    plots += [
        df.hvplot.scatter(x=f"unique_{k}", y=k, xlabel="sketch size", hover_cols=["id"])
        * df.hvplot.line(x=f"unique_{k}", y=f"constant_{k}", color="orange")
    ]
reduce(operator.add, plots).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Scatter.I :Scatter   [unique_21]   (21,id)
      .Curve.I   :Curve   [unique_21]   (constant_21)
   .Overlay.II  :Overlay
      .Scatter.I :Scatter   [unique_31]   (31,id)
      .Curve.I   :Curve   [unique_31]   (constant_31)
   .Overlay.III :Overlay
      .Scatter.I :Scatter   [unique_51]   (51,id)
      .Curve.I   :Curve   [unique_51]   (constant_51)